# Training of Basic, Maxpool, Temp, LSTM and Mix CNN Models on EEG Data

##Imports

In [ ]:
import numpy as np 
import scipy.io as sio
import torch
import os 

import torch.optim as optim
import torch.nn.functional as F

from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader,random_split

In [ ]:
from Utils import *
from Models import *

torch.manual_seed(1234)
np.random.seed(1234)

import warnings
warnings.simplefilter("ignore")

## Loading Images 
The images are loaded from .mat files for easier processing. The numpy arrays store the attributes of the dataset, as listed below.

In [ ]:
#Loads a mean image sampled over all seven windows of the EEG scan
Mean_Images = sio.loadmat("./Sample Data/images.mat")["img"] 
print(np.shape(Mean_Images))

#Loads the images correpsonding to the 7-window EEG scan
Images = sio.loadmat("./Sample Data/images.mat")["img"]
print(np.shape(Images))

#Denotes signal/load labels for each set of scans/mean image
Label = (sio.loadmat("./Sample Data/FeatureMat_timeWin")["features"][:,-1]-1).astype(int)
print(np.shape(Label)) 

#Denotes Patient IDs
Patient_id = sio.loadmat("./Sample Data/trials_subNums.mat")['subjectNum'][0]
print(np.shape(Patient_id))

(2670, 3, 32, 32)
(2670, 3, 32, 32)
(2670,)
(2670,)


## Loading patient dataset 
From the total data, we select the images corresponding patient. 

In [ ]:
print("Choose among the patient : "+str(np.unique(Patient_id)))

Choose among the patient : [ 1  2  3  4  6  7  8  9 10 11 12 14 15]


In [ ]:
chosen_patient = 9

In [ ]:
train_split = 0.8
test_split = 0.2
batch_size = 32

In [ ]:
EEG = EEGImagesDataset(label=Label[Patient_id==chosen_patient], image=Mean_Images[Patient_id==chosen_patient])

lengths = [int(len(EEG)*train_split+1), int(len(EEG)*test_split)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

## Basic CNN
A basic CNN implementation on the image dataset.

In [ ]:
print('Begin Training for Patient '+str(chosen_patient))
res = TrainTest_Model(BasicCNN, Trainloader, Testloader, n_epoch=50, learning_rate=0.001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5,  50]	loss: 0.510	Accuracy : 0.796		val-loss: 0.391	val-Accuracy : 0.825
[10,  50]	loss: 0.097	Accuracy : 0.981		val-loss: 0.084	val-Accuracy : 0.950
[15,  50]	loss: 0.070	Accuracy : 0.963		val-loss: 0.080	val-Accuracy : 0.950
[20,  50]	loss: 0.030	Accuracy : 0.981		val-loss: 0.016	val-Accuracy : 1.000
[25,  50]	loss: 0.001	Accuracy : 1.000		val-loss: 0.038	val-Accuracy : 0.975
[30,  50]	loss: 0.000	Accuracy : 1.000		val-loss: 0.019	val-Accuracy : 0.975
[35,  50]	loss: 0.000	Accuracy : 1.000		val-loss: 0.017	val-Accuracy : 0.975
[40,  50]	loss: 0.000	Accuracy : 1.000		val-loss: 0.016	val-Accuracy : 0.975
[45,  50]	loss: 0.000	Accuracy : 1.000		val-loss: 0.014	val-Accuracy : 0.975
[50,  50]	loss: 0.000	Accuracy : 1.000		val-loss: 0.009	val-Accuracy : 1.000


## Maxpool CNN
A Max-pooling model which performs a maxpool over the 7 EEG windows.

In [ ]:
EEG = EEGImagesDataset(label=Label[Patient_id==chosen_patient], image=Images[Patient_id==chosen_patient])

lengths = [int(len(EEG)*train_split+1), int(len(EEG)*test_split)]
Train, Test = random_split(EEG, lengths)

Trainloader = DataLoader(Train,batch_size=batch_size)
Testloader = DataLoader(Test, batch_size=batch_size)

In [ ]:
print('Begin Training for Patient '+str(chosen_patient))
res = TrainTest_Model(MaxCNN, Trainloader, Testloader, n_epoch=50, learning_rate=0.001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5,  45]	loss: 0.850	Accuracy : 0.617		val-loss: 0.644	val-Accuracy : 0.875
[10,  45]	loss: 0.159	Accuracy : 0.920		val-loss: 0.980	val-Accuracy : 0.825
[15,  45]	loss: 0.094	Accuracy : 0.963		val-loss: 0.233	val-Accuracy : 0.900
[20,  45]	loss: 0.003	Accuracy : 1.000		val-loss: 0.090	val-Accuracy : 0.950
[25,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.017	val-Accuracy : 1.000
[30,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.017	val-Accuracy : 1.000
[35,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.015	val-Accuracy : 1.000
[40,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.011	val-Accuracy : 1.000
[45,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 0.006	val-Accuracy : 1.000


## Temp CNN
A Conv1D model which performs a 1D convolution over the 7 EEG windows.

In [ ]:
print('Begin Training for Patient '+str(chosen_patient))
res = TrainTest_Model(TempCNN, Trainloader, Testloader, n_epoch=50, learning_rate=0.001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5,  45]	loss: 0.438	Accuracy : 0.821		val-loss: 1.963	val-Accuracy : 0.750
[10,  45]	loss: 0.153	Accuracy : 0.938		val-loss: 2.079	val-Accuracy : 0.875
[15,  45]	loss: 0.005	Accuracy : 1.000		val-loss: 4.025	val-Accuracy : 0.825
[20,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.416	val-Accuracy : 0.900
[25,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.587	val-Accuracy : 0.900
[30,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.672	val-Accuracy : 0.900
[35,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 3.827	val-Accuracy : 0.900
[40,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 4.332	val-Accuracy : 0.900
[45,  45]	loss: 0.000	Accuracy : 1.000		val-loss: 5.142	val-Accuracy : 0.900
Finished Training 
 loss: 0.000	Accuracy : 1.000		val-loss: 5.142	val-Accuracy : 0.900


## LSTM CNN
An LSTM Model which applies an RNN over the 7 EEG windows.

In [ ]:
print('Begin Training for Patient '+str(chosen_patient))
res = TrainTest_Model(LSTM, Trainloader, Testloader, n_epoch=50, learning_rate=0.0001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5,  45]	loss: 1.321	Accuracy : 0.333		val-loss: 1.386	val-Accuracy : 0.225
[10,  45]	loss: 1.225	Accuracy : 0.364		val-loss: 1.427	val-Accuracy : 0.325
[15,  45]	loss: 1.002	Accuracy : 0.562		val-loss: 1.115	val-Accuracy : 0.450
[20,  45]	loss: 0.664	Accuracy : 0.704		val-loss: 0.744	val-Accuracy : 0.600
[25,  45]	loss: 0.392	Accuracy : 0.895		val-loss: 0.515	val-Accuracy : 0.775
[30,  45]	loss: 0.187	Accuracy : 0.988		val-loss: 0.285	val-Accuracy : 0.900
[35,  45]	loss: 0.082	Accuracy : 1.000		val-loss: 0.236	val-Accuracy : 0.825
[40,  45]	loss: 0.040	Accuracy : 1.000		val-loss: 0.170	val-Accuracy : 0.900
[45,  45]	loss: 0.022	Accuracy : 1.000		val-loss: 0.141	val-Accuracy : 0.950
Finished Training 
 loss: 0.022	Accuracy : 1.000		val-loss: 0.141	val-Accuracy : 0.950


## Mix CNN
An LSTM Model which applies both a CNN and an RNN over the 7 EEG windows.

In [ ]:
print('Begin Training for Patient '+str(chosen_patient))
res = TrainTest_Model(Mix, Trainloader, Testloader, n_epoch=50, learning_rate=0.00001, print_epoch=5, opti='Adam')

Begin Training for Patient 9
[5, 100]	loss: 1.375	Accuracy : 0.296		val-loss: 1.376	val-Accuracy : 0.150
[10, 100]	loss: 1.362	Accuracy : 0.302		val-loss: 1.371	val-Accuracy : 0.225
[15, 100]	loss: 1.352	Accuracy : 0.302		val-loss: 1.368	val-Accuracy : 0.225
[20, 100]	loss: 1.342	Accuracy : 0.302		val-loss: 1.362	val-Accuracy : 0.225
[25, 100]	loss: 1.312	Accuracy : 0.302		val-loss: 1.335	val-Accuracy : 0.225
[30, 100]	loss: 1.191	Accuracy : 0.302		val-loss: 1.250	val-Accuracy : 0.225
[35, 100]	loss: 0.981	Accuracy : 0.586		val-loss: 1.105	val-Accuracy : 0.575
[40, 100]	loss: 0.836	Accuracy : 0.605		val-loss: 1.015	val-Accuracy : 0.650
[45, 100]	loss: 0.760	Accuracy : 0.611		val-loss: 1.008	val-Accuracy : 0.700
[50, 100]	loss: 0.677	Accuracy : 0.654		val-loss: 1.047	val-Accuracy : 0.725
[55, 100]	loss: 0.561	Accuracy : 0.753		val-loss: 1.120	val-Accuracy : 0.725
[60, 100]	loss: 0.421	Accuracy : 0.833		val-loss: 1.253	val-Accuracy : 0.800
[65, 100]	loss: 0.301	Accuracy : 0.895		val-loss